In [ ]:
import pandas as pd
import numpy as np
import MySQLdb
import time, socket, sys, os
from datetime import datetime

In [ ]:
def view_table(name):
    cursor.execute(f"SELECT * FROM {name}")
    myresult = cursor.fetchall()
    for x in myresult:
        print(x)

### Import Player

In [ ]:
df = pd.read_csv("player_info.csv", index_col=0, header=0)
df = df.replace(np.nan, '', regex=True)

In [ ]:
database = MySQLdb.connect(host="dbase.cs.jhu.edu", user="20fa_rzhai2", passwd="AgadM8or", db="20fa_rzhai2_db")
cursor = database.cursor()
query = "INSERT INTO Player(RealName, Birthday, Country, Status, Team, Earning, Role, SignatureHero, PlayerName) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [ ]:
def rebuild_Player_table():
    cursor.execute("DROP TABLE Player")
    definition = (f"CREATE TABLE Player (RealName VARCHAR(30),Birthday VARCHAR(20),Country VARCHAR(40)," 
                 f"Status VARCHAR(10),"
                 f"Team VARCHAR(30),Earning VARCHAR(20),Role VARCHAR(30),SignatureHero VARCHAR(70)," 
                 f"PlayerName VARCHAR(50)  Primary Key)")
    cursor.execute(definition)

In [ ]:
#cursor.execute("TRUNCATE TABLE Player")
for row in df.itertuples():
    cursor.execute(query,
                (row._1, f"{row.Birth} 00:00:00", row.Country.strip(), row.Status, row.Team, row._6, row._7, row._8, row._9))
print(f"{len(list(df.itertuples()))} entries inserted")
database.commit()
cursor.close()
database.close()

### Import Team

In [ ]:
df = pd.read_csv("team_info.csv", index_col=0, header=0)
df = df.replace(np.nan, '', regex=True)
list(df.itertuples())[0]

In [ ]:
database = MySQLdb.connect(host="dbase.cs.jhu.edu", user="20fa_rzhai2", passwd="AgadM8or", db="20fa_rzhai2_db")
cursor = database.cursor()
query = "INSERT INTO Team(City, Country, TeamName) VALUES (%s, %s, %s)"

In [ ]:
#cursor.execute("TRUNCATE TABLE Team")
for row in df.itertuples():
    cursor.execute(query,
                (row._1.strip(), row.Location.strip(), row._3.strip()))
print(f"{len(list(df.itertuples()))} entries inserted")
database.commit()
cursor.close()
database.close()

### Import ServesIn

In [ ]:
df = pd.read_csv("player_history.csv", index_col=0, header=0)
df = df.replace(np.nan, '', regex=True)
list(df.itertuples())[0]

In [ ]:
database = MySQLdb.connect(host="dbase.cs.jhu.edu", user="20fa_rzhai2", passwd="AgadM8or", db="20fa_rzhai2_db")
cursor = database.cursor()
query = "INSERT INTO ServesIn(PlayerName, StartDate, EndDate, TeamName) VALUES (%s, %s, %s, %s)"

In [ ]:
def rebuild_ServesIn_table():
    cursor.execute("DROP TABLE ServesIn")
    definition = (f"CREATE TABLE ServesIn(PlayerName VARCHAR(50), StartDate DATE," 
                 f"EndDate DATE, TeamName VARCHAR(50), PRIMARY KEY (PlayerName, Startdate, EndDate, TeamName))")
    cursor.execute(definition)
    print("Re-built")
    
#rebuild_ServesIn_table()

In [ ]:
#cursor.execute("TRUNCATE TABLE ServesIn")
for row in df.itertuples():
    start = row.start_time.replace('?', '0') 
    start = "" if start.replace('0', '') == '' else start
    end   = row.end_time.replace('?', '0') 
    end   = "" if end.replace('0', '') == '' else end
    if row.end_time.lower().strip() == 'present':
        end = datetime.utcnow().strftime('%Y-%m-%d')
    cursor.execute(query,
                (row._1.strip(), f"{start} 00:00:00", f"{end} 00:00:00", row.Team.strip()))
    
print(f"{len(list(df.itertuples()))} entries inserted")
#view_table("ServesIn")
database.commit()
cursor.close()
database.close()

### Import Match_Map_Stats

In [ ]:
df = pd.read_csv("match_map_stats.csv", index_col=None, header=0)
df = df.replace(np.nan, -1, regex=True)
print(df.shape)
df = df.drop_duplicates(subset=['round_start_time', 'round_end_time', 'stage', 'match_id'])
print(df.shape)

In [ ]:
database = MySQLdb.connect(host="dbase.cs.jhu.edu", user="20fa_rzhai2", passwd="AgadM8or", db="20fa_rzhai2_db")
cursor = database.cursor()
query = (f"INSERT INTO MatchMapStats(round_start_time, round_end_time, stage, match_id, "
         f"game_number, match_winner, map_winner, map_loser, map_name, "
         f"map_round, winning_team_final_map_score, losing_team_final_map_score, "
         f"control_round_name, attacker, defender, team_one_name, team_two_name, "
         f"attacker_payload_distance, defender_payload_distance, attacker_time_banked, "
         f"defender_time_banked, attacker_control_perecent, defender_control_perecent, "
         f"attacker_round_end_score, defender_round_end_score)"
         f" VALUES (%s, %s, %s, %s, "
         f"%s, %s, %s, %s, %s, %s, %s, %s, "
         f"%s, %s, %s, %s, %s, %s, %s, %s, "
         f"%s, %s, %s, %s, %s)")

In [ ]:
def rebuild_MatchMapStats_table():
    cursor.execute("DROP TABLE MatchMapStats")
    definition = (f"CREATE TABLE MatchMapStats(round_start_time DATETIME, round_end_time DATETIME, stage VARCHAR(50),"
                  f"match_id MEDIUMINT(10), game_number TINYINT(10), match_winner VARCHAR(22), map_winner VARCHAR(22),"
                  f"map_loser VARCHAR(22), map_name VARCHAR(25), map_round TINYINT(10),"
                  f"winning_team_final_map_score TINYINT(10), losing_team_final_map_score TINYINT(10),"
                  f"control_round_name VARCHAR(25), attacker VARCHAR(22), defender VARCHAR(22),"
                  f"team_one_name VARCHAR(22), team_two_name VARCHAR(22), attacker_payload_distance FLOAT(40, 20),"
                  f"defender_payload_distance FLOAT(28, 20), attacker_time_banked FLOAT(28, 20),"
                  f"defender_time_banked FLOAT(28, 20), attacker_control_perecent FLOAT(28, 20),"
                  f"defender_control_perecent FLOAT(28, 20), attacker_round_end_score TINYINT(10),"
                  f"defender_round_end_score TINYINT(10), PRIMARY KEY (round_start_time, round_end_time, stage, match_id))")
    cursor.execute(definition)
    print("Re-built")
    
#rebuild_MatchMapStats_table()

In [1]:
#cursor.execute("TRUNCATE TABLE MatchMapStats")
c = 0
for row in df.itertuples():
    if c % 100 == 0:
        print(c)
    round_start_time = '0000-00-00 00:00:00' if row.round_start_time == -1 else row.round_start_time
    round_end_time = '0000-00-00 00:00:00' if row.round_end_time == -1 else row.round_end_time     
        
    values = (round_start_time, round_end_time, row.stage, row.match_id, 
              row.game_number, row.match_winner, row.map_winner, row.map_loser, row.map_name,
              row.map_round, row.winning_team_final_map_score, row.losing_team_final_map_score,
              row.control_round_name, row.attacker, row.defender, row.team_one_name, row.team_two_name,
              row.attacker_payload_distance, row.defender_payload_distance, row.attacker_time_banked,
              row.defender_time_banked, row.attacker_control_perecent, row.defender_control_perecent,
              row.attacker_round_end_score, row.defender_round_end_score)
    cursor.execute(query, values)
    c += 1
    
print(f"{len(list(df.itertuples()))} entries inserted")
view_table("MatchMapStats")
database.commit()
cursor.close()
database.close()

NameError: name 'df' is not defined

### Import GameInfo

In [ ]:
r = []
for i in [2020]:
    basedir = os.path.join(os.getcwd(), f'phs_{i}', '')
    for f in os.listdir(basedir):
        df = pd.read_csv(basedir + f, )
        df = df.drop_duplicates(subset=["esports_match_id", "map_name", "player_name", "hero_name", "stat_name"])
        #df = df.drop_duplicates()
        if len(r) == 0:
            r = df
        else:
            r = pd.concat([r, df])
            r = r.drop_duplicates(subset=["esports_match_id", "map_name", "player_name", "hero_name", "stat_name"])



In [ ]:
r.shape

In [ ]:
r = r.replace(np.nan, -1, regex=True)

In [ ]:
database = MySQLdb.connect(host="dbase.cs.jhu.edu", user="20fa_rzhai2", passwd="AgadM8or", db="20fa_rzhai2_db")
cursor = database.cursor()
query = (f"INSERT INTO 2020Stats(start_time, esports_match_id,"
         f" tournament_title, map_type, map_name, player_name,"
         f" team_name, stat_name, hero_name, stat_amount)"
         f" VALUES"
         f" (%s, %s, %s, %s, "
         f" %s, %s, %s, %s, %s, %s)")

In [ ]:
def rebuild_2020_table():
    #cursor.execute("DROP TABLE 2020Stats")
    definition = (f"CREATE TABLE 2020Stats(start_time DATETIME, esports_match_id MEDIUMINT(10),"
                  f" tournament_title VARCHAR(24), "
                  f" map_type VARCHAR(8), map_name VARCHAR(22), player_name VARCHAR(11),"
                  f" team_name VARCHAR(23), stat_name VARCHAR(42),"
                  f"hero_name VARCHAR(13), stat_amount FLOAT(30, 20),"
                  f" PRIMARY KEY (esports_match_id, map_name, player_name, hero_name, stat_name))")
    cursor.execute(definition)
    print("Re-built")
    
#rebuild_2020_table()

In [ ]:
#cursor.execute("TRUNCATE TABLE 2020Stats")
c = 0
m = list(r.itertuples())

In [ ]:
for idx in range(c, len(m)):
    row = m[idx]
    if c % 5000 == 0:
        print(c)
    tmp = row.start_time.split(' ')
    """ For first part of csv """
    #date = tmp[0].split('/')
    #start_t = '-'.join([date[2], date[0], date[1]]) + " " + tmp[1]
    """ For second part of csv """
    start_t = row.start_time
    
    values = (start_t, row.esports_match_id,row.tournament_title,
              row.map_type,row.map_name,row.player_name,row.team_name,
              row.stat_name,row.hero_name,row.stat_amount)
    cursor.execute(query, values)
    c += 1
    
print(f"{len(list(df.itertuples()))} entries inserted")
view_table("2020Stats")
database.commit()
cursor.close()
database.close()

In [ ]:
# To Check how many entries ran
c

In [ ]:
# To see how many entries exist in 2020Stats Table
database = MySQLdb.connect(host="dbase.cs.jhu.edu", user="20fa_rzhai2", passwd="AgadM8or", db="20fa_rzhai2_db")
cursor = database.cursor()

cursor.execute(f"SELECT * FROM 2020Stats")
myresult = cursor.fetchall()
print(len(list(myresult)))